In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

# This is done so that the notebook can see files
# placed in a different folder
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import matplotlib.image as mpimg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import osmnx as ox

import functions as F

## Data collection

### Voting data

Data for the 2020 local elections, for each locality.

In [47]:
import requests
import time
import json

coduri = ['ab', 'ar', 'ag', 'bc', 'bh', 'bn', 'bt', 'br', 'bv', 'bz', 'cl', 'cs', 'cj', 'ct', 'cv', 'db', 
          'dj', 'gl', 'gr', 'gj', 'hr', 'hd', 'il', 'is', 'if', 'mm', 'mh', 'ms', 'nt', 'ot', 
          'ph', 'sj', 'sm', 'sb', 'sv', 'tr', 'tm', 'tl', 'vl', 'vs', 'vn', 'b']

res_list = []

for cod in coduri:
    url = f'https://prezenta.roaep.ro/locale27092020/data/json/sicpv/pv/pv_{cod}_final.json?_={time.time_ns() // 1_000_000}'
    r = requests.get(url)
    print(f'fetched {cod}')
    json_data = json.loads(r.text)
    results = json_data['stages']['FINAL']['scopes']['UAT']['categories']['P']['table']
    for k in results:
        try:
            top_candidate = F.get_top_candidate(results[k])
            res_list.append(top_candidate)
        except ValueError:
            continue

vote_data = pd.DataFrame(res_list)

fetched ab
fetched ar
fetched ag
fetched bc
fetched bh
fetched bn
fetched bt
fetched br
fetched bv
fetched bz
fetched cl
fetched cs
CARAŞ-SEVERIN ORAŞ MOLDOVA NOUĂ
fetched cj
fetched ct
fetched cv
fetched db
fetched dj
fetched gl
fetched gr
fetched gj
fetched hr
fetched hd
fetched il
fetched is
fetched if
fetched mm
fetched mh
fetched ms
fetched nt
fetched ot
fetched ph
fetched sj
fetched sm
fetched sb
fetched sv
fetched tr
fetched tm
fetched tl
fetched vl
fetched vs
fetched vn
fetched b


In [2]:
# vote_data = vote_data.append(pd.DataFrame.from_dict({'partid':['MIX'], 'localitate':['MUNICIPIUL BUCURESTI'], 'siruta':[179132], 'judet':['bucuresti']}))
# vote_data['localitate'] = vote_data['localitate'].apply(lambda x: F.normalize(x, strip_city=False))
# vote_data['judet'] = vote_data['judet'].apply(lambda x: F.normalize(x, True))
# vote_data.to_csv("vote_data_2020_primar.csv", sep=",", header=True, index=False)
vote_data = pd.read_csv("vote_data_2020_primar.csv")

### Vaccination data
Updated with 5th of August data

In [3]:
from tika import parser
import re

raw = parser.from_file('AV-pe-uat-5.10.2021.pdf')
p = re.compile('(.*)(\s[A-Z]{1}\s)(.*)(\s\d{1,2}\.\d{1,2})')
records = []

counter = 0

for line in raw['content'].split('\n'):
    if len(line) > 10:
        entry_dict = {}
        match = p.match(line)
        if match is None:
            continue
        
        localitate = match.group(3).strip()        
#         print(localitate)
        if not counter % 100:
            print(counter)
        counter += 1
        
        tip_uat = match.group(2).strip()
        judet = match.group(1).strip()
        if tip_uat == 'M':
            # Strip 'MUNICIPIUL' from big city names.
            localitate = localitate[11:]
        if tip_uat == 'O':
            # Strip 'MUNICIPIUL' from big city names.
            localitate = localitate[5:]
        if judet == "MUNICIPIUL BUCUREŞTI":
            localitate = "BUCURESTI"

#         try:
#             loc_gdf = ox.graph_from_place(f'{localitate}, {judet}, Romania', which_result=0)
#         except IndexError:
#             print("No gdf for " + localitate)
#             continue
#         lat, long = F.get_middle_coords(loc_gdf)
        
        entry_dict['localitate'] = localitate
        entry_dict['judet'] = judet
        entry_dict['tip_uat'] = tip_uat
        entry_dict['procent_vacc'] = match.group(4).strip()
#         entry_dict['lat'] = lat
#         entry_dict['long'] = long
        records.append(entry_dict)

vacc_data = pd.DataFrame.from_records(records)

2021-10-15 18:43:06,668 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar to /tmp/tika-server.jar.
2021-10-15 18:43:13,823 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar.md5 to /tmp/tika-server.jar.md5.
2021-10-15 18:43:14,402 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100


In [7]:
vacc_data['judet'] = vacc_data['judet'].apply(lambda x: F.normalize(x, True))
vacc_data['localitate'] = vacc_data['localitate'].apply(lambda x: F.normalize(x, False))

In [3]:
# vacc_data['judet'] = vacc_data['judet'].apply(lambda x: F.normalize(x, True))
# vacc_data = vacc_data[:3181]
# vacc_data = vacc_data.astype({'procent_vacc': 'float64'})
# vacc_data.to_csv("vacc_data_5_aug_2021_cu_judet.csv", sep=",", header=True, index=False)
vacc_data = pd.read_csv('vacc_data_5_oct_2021_cu_judet.csv')

### Vaccination locations

In [69]:
locations = pd.read_csv('centre_covid.csv')

In [5]:
vacc_data['min_dist'] = 100000
vacc_data['nearest_count'] = 0
vacc_data['nearest_std'] = 0

for i in range(vacc_data.shape[0]):
    min_dist, nc, nsd = F.nearest_centers(vacc_data.iloc[i]['lat'], 
                                        vacc_data.iloc[i]['long'], locations)
    vacc_data.at[i, 'min_dist'] = min_dist
    vacc_data.at[i, 'nearest_count'] = nc
    vacc_data.at[i, 'nearest_std'] = nsd
    

AttributeError: module 'osmnx' has no attribute 'great_circle_vec'

#### Use Google cloud API to get location elevations

In [ ]:
step = 100
start_idxs = [i for i in range(0, vacc_data.shape[0], step)] + [vacc_data.shape[0]]
all_elevations = []
for i in range(len(start_idxs)-1):
    print(f'{i} {start_idxs[i]} {start_idxs[i+1]}')
    response = get_elevations(vacc_data, start_idxs[i], start_idxs[i+1], '[API key here]')
    all_elevations += elevations_from_response(response)

### Age data

In [4]:
age_data = pd.read_csv('sR_Tab_31.csv')

In [5]:
age_data = F.add_county(age_data)

age_data = F.filter_rows(age_data)
age_data = age_data.rename(columns={'uat':'localitate'})

age_data['localitate'] = age_data['localitate'].apply(lambda x: F.normalize(x, strip_city=False))
age_data['judet'] = age_data['judet'].apply(lambda x: F.normalize(x, True))

age_data['over65_ratio'] = (age_data['65_69'] + age_data['70_74'] + age_data['75_79'] +
                            age_data['80_84'] + age_data['peste_85']) / age_data['total']

age_data['over60_ratio'] = (age_data['60_64'] + age_data['65_69'] + age_data['70_74'] + age_data['75_79'] +
                            age_data['80_84'] + age_data['peste_85']) / age_data['total']

age_data['under40_ratio'] = (age_data['sub_5'] + age_data['5_9'] + age_data['10_14'] + 
                             age_data['15_19'] + age_data['20_24'] + age_data['25_29'] +
                             age_data['30_34'] + age_data['35_39']) / age_data['total']

### Education data

In [6]:
ed_data = pd.read_csv('sR_TAB_161.csv')

In [7]:
ed_data = F.add_county(ed_data)
ed_data = F.filter_rows(ed_data)
ed_data = ed_data.rename(columns={'uat':'localitate'})
ed_data['localitate'] = ed_data['localitate'].apply(lambda x: F.normalize(x, strip_city=False))
ed_data['judet'] = ed_data['judet'].apply(lambda x: F.normalize(x, True))

ed_data['tertiary_ratio'] = ed_data['total_high'] / ed_data['total_peste10']
ed_data['illiterate_ratio'] = (ed_data['no_school'] + ed_data['illiterate']) / ed_data['total_peste10']

### Religion data

In [14]:
rel_data = pd.read_csv('sR_TAB_13.csv')
rel_data['sex'] = 'Ambele sexe'
rel_data = rel_data.fillna(0)
rel_data = F.add_county(rel_data, step=1, start=0)
rel_data = F.filter_rows(rel_data)
rel_data = rel_data.rename(columns={'uat':'localitate'})
rel_data['localitate'] = rel_data['localitate'].apply(lambda x: F.normalize(x, strip_city=False))
rel_data['judet'] = rel_data['judet'].apply(lambda x: F.normalize(x, True))

In [15]:
rel_data['orthodox_ratio'] = (rel_data['ortodoxa'] + rel_data['crestina_rit_vechi'] + 
                              rel_data['ortodoxa_sarba'] + rel_data['armeana']) / rel_data['total']
rel_data['catholic_ratio'] = (rel_data['romano_catolica'] + rel_data['greco_catolica']) / rel_data['total']
rel_data['protestant_ratio'] = (rel_data['reformata'] + rel_data['lutherana'] + rel_data['unitariana'] + 
                                rel_data['evanghelica'] + rel_data['evanghelica_augustana']) / rel_data['total']
rel_data['neoprot_ratio'] = (rel_data['penticostala'] + rel_data['baptista'] + 
                             rel_data['crestina_dupa_evanghelie'] +
                             rel_data['adventista_z7'] + rel_data['martorii_lui_iehova']) / rel_data['total']
rel_data['other_ratio'] = (rel_data['musulmana'] + rel_data['mozaica'] + 
                           rel_data['alta_religie']) / rel_data['total']
rel_data['irreligious_ratio'] = (rel_data['fara_religie'] + rel_data['atei']) / rel_data['total']
rel_data['na_ratio'] = rel_data['na'] / rel_data['total']

### Joining data

In [16]:
ed_data = ed_data.set_index(['localitate', 'judet'])
age_data = age_data.set_index(['localitate', 'judet'])
vote_data = vote_data.set_index(['localitate', 'judet'])
vacc_data = vacc_data.set_index(['localitate', 'judet'])
rel_data = rel_data.set_index(['localitate', 'judet'])

age_ed = ed_data[['illiterate_ratio', 'tertiary_ratio']].join(age_data[['over65_ratio', 'under40_ratio', 'total']])

age_ed_rel_gr = age_ed.join(rel_data[['orthodox_ratio','catholic_ratio','protestant_ratio',
                                      'neoprot_ratio','other_ratio','irreligious_ratio','na_ratio']])

age_ed_vote_gr = age_ed_rel_gr.join(vote_data)
age_ed_vote_gr = age_ed_vote_gr.reset_index()
age_ed_vote_gr['localitate'] = age_ed_vote_gr['localitate'].apply(lambda x: F.normalize(x, strip_city=True))

age_ed_vote_gr = age_ed_vote_gr.set_index(['localitate', 'judet'])


age_ed_vote_vacc_gr = age_ed_vote_gr.join(vacc_data)

In [17]:
age_ed_vote_vacc_gr = age_ed_vote_vacc_gr.drop(age_ed_vote_vacc_gr[age_ed_vote_vacc_gr['procent_vacc'].isnull()].index)

In [18]:
age_ed_vote_vacc_gr = age_ed_vote_vacc_gr.reset_index()

In [19]:
age_ed_vote_vacc_gr.to_csv('vacc_vote_age_ed_rel_data.csv', sep=",", header=True, index=False)

### Covid rates

In [86]:
covid_data = pd.read_csv('covid_rate_14oct21_over3.csv')

In [89]:
covid_data['judet'] = covid_data['judet'].apply(lambda x: F.normalize(x, True))
covid_data['localitate'] = covid_data['localitate'].apply(lambda x: F.normalize(x, False))

In [91]:
covid_data.set_index(['localitate', 'judet'], inplace=True)

In [93]:
age_ed_vote_vacc_gr.set_index(['localitate', 'judet'], inplace=True)

In [95]:
covid_data = covid_data.join(age_ed_vote_vacc_gr, how='inner')

ValueError: columns overlap but no suffix specified: Index(['illiterate_ratio', 'tertiary_ratio', 'over65_ratio', 'under40_ratio',
       'total', 'partid', 'siruta', 'lat', 'long', 'min_dist', 'nearest_count',
       'nearest_std', 'alt', 'tip_uat', 'procent_vacc'],
      dtype='object')

In [104]:
covid_data.reset_index(inplace=True)

In [105]:
covid_data.to_csv('all_with_covid_rate_14oct21.csv',  sep=",", header=True, index=False)

In [106]:
covid_data[covid_data['incidenta'] >= 15][['populatie', 'cazuri', 'incidenta', 'procent_vacc']]

,populatie,cazuri,incidenta,procent_vacc
0,31000,627,20.23,39.31
1,45843,827,18.04,35.70
2,20542,369,17.96,44.46
3,12597,213,16.91,32.32
4,11155,188,16.85,24.39
5,2205227,34010,15.42,38.56
250,1000,26,26.00,12.48
251,7600,180,23.68,30.17
252,1106,25,22.60,10.05
253,9516,211,22.17,43.08
